In [2]:
import pandas as pd
import json
import requests
import datetime

In [3]:
def sjs_calender(month_str):
    url='http://www.szse.cn/api/report/exchange/onepersistenthour/monthList?month={}'.format(month_str)
    res=requests.get(url)
    res_data=json.loads(res.text)['data']
    df=pd.DataFrame(res_data).set_index('jyrq')
    df.columns=['weekday','trade_day']
    return df

def get_next_tradeday(date_str:str):
    """

    :param date_str: 传入一个完整的时间字符串如‘2021-02-02’
    :return:
    """
    year=int(date_str[0:4])
    month=int(date_str[5:7])
    if month != 12:
        next_month='{}-{}'.format(year,month+1)
    else:
        next_month='{}-{}'.format(year+1,'01')
    df=pd.concat([sjs_calender(date_str[:7]),sjs_calender(next_month)])
    after_today = df.loc[date_str:].iloc[1:]
    next_trade_day= after_today[after_today['trade_day']=='1'].index[0]
    return next_trade_day


In [6]:
get_next_tradeday('2020-12-01')

'2020-12-02'

In [27]:
def sjs_calender(month_str):
    url='http://www.szse.cn/api/report/exchange/onepersistenthour/monthList?month={}'.format(month_str)
    res=requests.get(url)
    res_data=json.loads(res.text)['data']
    df=pd.DataFrame(res_data)
    df.columns=['weekday','trade_day','calendar_day']
    df.set_index('calendar_day',inplace=True)
    return df


In [28]:
pd.concat([sjs_calender(str(2021)+'-'+str(month).zfill(2)) for month in range(1,13)])


,weekday,trade_day
calendar_day,,
2021-01-01,6,0
2021-01-02,7,0
2021-01-03,1,0
2021-01-04,2,1
2021-01-05,3,1
...,...,...
2021-12-27,2,1
2021-12-28,3,1
2021-12-29,4,1


In [30]:
from sqlalchemy import create_engine

trade_day_db_path='../my_scheduled_app/'+'交易日.db'
engine_trade_day = create_engine(r"sqlite:///"+trade_day_db_path)

for year in range(2005,2022):
    print(year)
    year_all=pd.concat([sjs_calender(str(year)+'-'+str(month).zfill(2)) for month in range(1,13)])
#     print(year_all)
    year_all.to_sql('year'+str(year), engine_trade_day, if_exists="replace")
    

2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021


In [83]:
def sjs_calender_year(year): #深交所
    url='http://www.szse.cn/api/report/exchange/onepersistenthour/monthList?month={}-{}'

    df_list=[]
    for month in range(1,13):
        month_url=url.format(year,str(month).zfill(2))
        res=requests.get(month_url)
        res_data=json.loads(res.text)['data']
        df=pd.DataFrame(res_data)
        df_list.append(df)
    df_year=pd.concat(df_list)
    df_year.columns=['weekday','trade_day','calendar_day']
    df_year.set_index('calendar_day',inplace=True)

    return df_year

def get_next_tradeday(date_str:str):
    """

    :param date_str: 传入一个完整的时间字符串如‘2021-02-02’
    :return:
    """
    year=int(date_str[0:4])
    month=int(date_str[5:7])
    if month != 12:
        next_month='{}-{}'.format(year,month+1)
    else:
        next_month='{}-{}'.format(year+1,'01')
    df=pd.read_sql('year'+str(year), engine_trade_day,index_col='calendar_day')
    after_today = df.loc[date_str:].iloc[1:]
    after_today_trade=after_today[after_today['trade_day']=='1']
    if after_today_trade.shape[0]==0:
        try: # 下一年存在
            print('选择下一年')
            after_today=pd.read_sql('year'+str(year+1), engine_trade_day,index_col='calendar_day')
        except: #下一年不存在，获取
            print('获取并选择下一年')
            year_all=sjs_calender_year(year+1)
            year_all.to_sql('year'+str(year+1), engine_trade_day, if_exists="replace")
            after_today=pd.read_sql('year'+str(year+1), engine_trade_day,index_col='calendar_day')
        after_today_trade=after_today[after_today['trade_day']=='1']

    next_trade_day= after_today_trade.index[0] # 今日后交易日的第一天
    return next_trade_day

In [84]:
# sjs_calender_year(2021)

In [85]:
get_next_tradeday('2020-12-31')

选择下一年


'2021-01-04'